Now we want to populate our data repository

In [1]:
import json
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import simplesimdb as simplesim # this one is our own database manager
import subprocess # to capture errors from the simulations
import itertools

In [2]:
m = simplesim.Manager(directory='advection', executable='./execute.sh', filetype='nc')

In [3]:
init2x= {"shear":[0.0,6.283185307179586],
         "mms": [-1.0,1.0]
         }
init2bc= {"shear":["PER", "DIR"],
          "mms": ["DIR", "PER"]}

init2init =  {
    "shear": {"type": "shear", "rho": 0.20943951023931953, "delta": 0.2 },
    "mms":   {"type": "mms", "sigma": 0.2, "velocity": 1.0}
}

n2v={
     "shear": {
    1:[144,1e-2],
    3:[48, 5e-3],
    4:[36, 2.5e-3],
    5:[28, 2e-3],
    7:[20, 1e-3]
    },
    "mms": {
    1:[144,2e-2],
    3:[48, 2e-3],
    4:[36, 2e-3],
    5:[28, 1e-3],
    7:[20, 5e-4]
    }
}

endtime = {"shear": 10, "mms": 1}

maxout = {"shear": 40, "mms": 5}
reg2reg = {"none" : {"type" : "none"},
           "modal" : {"type" : "modal", "alpha" : 36, "order" : 8, "eta_c" : 0.5},
          "viscous": {"type" : "viscosity", "order" : 1,
                      "nu" : 1e-3, "direction" : "forward"},
          "hyperviscous" : {"type": "viscosity", "order" : 2,
                            "nu" : 5e-4, "direction" : "forward"}
          }

In [4]:
def generate_standard_input( n, init, multiplication, advection, regularization) :
    itstp=endtime[init]/maxout[init]/n2v[init][n][1]
    if not itstp.is_integer() :
        print("Warning: itstp is not an integer!")
    inputfile={
        "grid":
        {   
            "type": "Cartesian2d",
            "n"  : n,
            "Nx" : n2v[init][n][0],
            "Ny" : n2v[init][n][0],
            "x": init2x[init],
            "y": init2x[init],
            "bc" : init2bc[init]
        },
        "timestepper":
        {   
            "type": "FilteredExplicitMultistep",
            "tableau": "eBDF-3-3",
            "dt" : n2v[init][n][1]
        },
        "regularization":  reg2reg[regularization],
        "output":
        {   
            "type": "netcdf",
            "maxout"  : int(maxout[init]),
            "tend" : int(endtime[init])
        },
        "advection":
        {   
            "multiplication": multiplication,
            "type": advection
        },
        "elliptic":
        {   
            "type" : "multigrid",
            "stages": 3,
            "eps_pol" : [1e-6,1.0,1.0],
            "direction" : "forward"
        },
        "init": init2init[init]
    }
    if regularization == "viscous" or regularization == "hyperviscous" :
        inputfile["timestepper"]["type"] = "ImExMultistep"
        inputfile["timestepper"]["tableau"] = "ImEx-BDF-3-3"
        inputfile["timestepper"]["eps_time"] = 1e-9
    return inputfile

In [5]:
###### THE RED BUTTON
#m.delete_all()

In [6]:
v_n=[1,3,4,5]
v_init=["shear", "mms"]
v_multiplication=["pointwise", "projection"]
v_advection=["arakawa","centered", "upwind"]
v_regularization=["none", "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4])   
    # these crashed on a first attempt
    if(( t == (3, 'shear', 'pointwise', 'centered', 'none')) 
        or (t==(3, 'shear', 'projection', 'arakawa', 'none'))
        or (t==(4, 'shear', 'pointwise', 'centered', 'none'))
        or (t==(4, 'shear', 'projection', 'arakawa', 'none'))
        or (t==(4, 'shear', 'projection', 'centered', 'none'))
        or (t==(5, 'shear', 'pointwise', 'arakawa', 'none'))
        or (t==(5, 'shear', 'projection', 'arakawa', 'none'))
        or (t==(5, 'shear', 'pointwise', 'centered', 'none'))
           ):
        print(t, " Crashes!! ")
        continue
    
#     #Check for existing data
#     if not m.exists(inputfile):
#         print(t)
#     uncomment the following to create data
    try:
        print(t)
        m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)
        break

(1, 'shear', 'pointwise', 'arakawa', 'none')
Existing simulation 17d30e...5588cd.nc
(1, 'shear', 'pointwise', 'arakawa', 'modal')
Existing simulation a49ca7...a8dc74.nc
(1, 'shear', 'pointwise', 'centered', 'none')
Existing simulation fc6fd6...228287.nc
(1, 'shear', 'pointwise', 'centered', 'modal')
Existing simulation b2dccf...22a117.nc
(1, 'shear', 'pointwise', 'upwind', 'none')
Existing simulation 8e3f10...0493db.nc
(1, 'shear', 'pointwise', 'upwind', 'modal')
Existing simulation 418212...513de0.nc
(1, 'shear', 'projection', 'arakawa', 'none')
Existing simulation a4fb02...1ac982.nc
(1, 'shear', 'projection', 'arakawa', 'modal')
Existing simulation 2dfdce...460c47.nc
(1, 'shear', 'projection', 'centered', 'none')
Existing simulation c3a726...21f12e.nc
(1, 'shear', 'projection', 'centered', 'modal')
Existing simulation b2c44f...3a2c9e.nc
(1, 'shear', 'projection', 'upwind', 'none')
Existing simulation 3b9fe8...48c4bf.nc
(1, 'shear', 'projection', 'upwind', 'modal')
Existing simulation

In [7]:
# change the eta_c parameter to 0 for comparison
v_n=[3,4,5]
v_init=["shear"]
v_multiplication=["pointwise","projection"]
v_advection=["arakawa","centered","upwind"]
v_regularization=[ "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4] ) 
    inputfile["regularization"]["eta_c"] = float(0)

    ncfile=''
    try:
        print(t)
        ncfile=m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)

(3, 'shear', 'pointwise', 'arakawa', 'modal')
Existing simulation 0053f6...290a99.nc
(3, 'shear', 'pointwise', 'centered', 'modal')
Existing simulation 7b6f2b...27fc4a.nc
(3, 'shear', 'pointwise', 'upwind', 'modal')
Existing simulation dd9523...85a9d4.nc
(3, 'shear', 'projection', 'arakawa', 'modal')
Existing simulation be6b5c...804c1e.nc
(3, 'shear', 'projection', 'centered', 'modal')
Existing simulation b34a3f...59d91b.nc
(3, 'shear', 'projection', 'upwind', 'modal')
Existing simulation c01c83...2803bd.nc
(4, 'shear', 'pointwise', 'arakawa', 'modal')
Existing simulation f7299a...facfe6.nc
(4, 'shear', 'pointwise', 'centered', 'modal')
Existing simulation 8279a1...f4b74f.nc
(4, 'shear', 'pointwise', 'upwind', 'modal')
Existing simulation 3626a9...adc2a3.nc
(4, 'shear', 'projection', 'arakawa', 'modal')
Existing simulation 3080cb...8ff758.nc
(4, 'shear', 'projection', 'centered', 'modal')
Existing simulation fdbed0...346522.nc
(4, 'shear', 'projection', 'upwind', 'modal')
Existing simu

In [8]:
# Generate one good simulation with double resolution
inputfile = generate_standard_input( 
        n=5, init="shear",multiplication="projection",
        advection="upwind",regularization="none" )
inputfile["grid"]["Nx"] *=2
inputfile["grid"]["Ny"] *=2
inputfile["timestepper"]["dt"]/=2.0

print( inputfile)
ncfile=''
try:
    ncfile=m.create(inputfile)
    #m.delete(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr)

{'grid': {'type': 'Cartesian2d', 'n': 5, 'Nx': 56, 'Ny': 56, 'x': [0.0, 6.283185307179586], 'y': [0.0, 6.283185307179586], 'bc': ['PER', 'DIR']}, 'timestepper': {'type': 'FilteredExplicitMultistep', 'tableau': 'eBDF-3-3', 'dt': 0.001}, 'regularization': {'type': 'none'}, 'output': {'type': 'netcdf', 'maxout': 40, 'tend': 10}, 'advection': {'multiplication': 'projection', 'type': 'upwind'}, 'elliptic': {'type': 'multigrid', 'stages': 3, 'eps_pol': [1e-06, 1.0, 1.0], 'direction': 'forward'}, 'init': {'type': 'shear', 'rho': 0.20943951023931953, 'delta': 0.2}}
Existing simulation 27b469...6c7fa6.nc


In [9]:
# Generate Simulations for first and second order diffusion on Arakawa scheme
v_nu = [1e-4, 2e-4, 5e-4, 1e-3, 2e-3]
v_order = [1,2]
v_n = [1,3,5]
for t in itertools.product( v_n, v_order, v_nu) :
    inputfile = generate_standard_input( 
        n=t[0], init="shear",multiplication="pointwise",
        advection="arakawa",regularization="viscous" )
    inputfile["regularization"]["order"] = t[1]
    inputfile["regularization"]["nu"] = t[2]
    if t[0] == 5 :
        inputfile["timestepper"]["dt"]/=2.0
    print(t)
    try:
        m.create(inputfile)
    except subprocess.CalledProcessError as e:
        print( e.stderr)


(1, 1, 0.0001)
Existing simulation f4205c...e14338.nc
(1, 1, 0.0002)
Existing simulation 293653...e3d2b0.nc
(1, 1, 0.0005)
Existing simulation 044abc...ad9b81.nc
(1, 1, 0.001)
Existing simulation c49b9d...0870a7.nc
(1, 1, 0.002)
Existing simulation fc7e2a...015693.nc
(1, 2, 0.0001)
Existing simulation 7265bd...e0d562.nc
(1, 2, 0.0002)
Existing simulation 5619c6...2da415.nc
(1, 2, 0.0005)
Existing simulation f27cdb...3148f3.nc
(1, 2, 0.001)
Existing simulation 6e1c20...b56831.nc
(1, 2, 0.002)
Existing simulation 1287e6...b9dd45.nc
(3, 1, 0.0001)
Existing simulation 36af49...608fc4.nc
(3, 1, 0.0002)
Existing simulation 05f86c...bcd68b.nc
(3, 1, 0.0005)
Existing simulation 0a2a20...c1e7cd.nc
(3, 1, 0.001)
Existing simulation 6e4b07...eb2736.nc
(3, 1, 0.002)
Existing simulation 2fccb5...d0df94.nc
(3, 2, 0.0001)
Existing simulation 5550fd...453d43.nc
(3, 2, 0.0002)
Existing simulation ccf055...db4c9a.nc
(3, 2, 0.0005)
Existing simulation 7eeabf...ae036e.nc
(3, 2, 0.001)
Existing simulation 

In [10]:
v_n=[1,3,4,5]
v_init=["shear"]
v_multiplication=["pointwise", "projection"]
v_advection=["upwind-advection","centered-advection"]
v_regularization=["none", "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4])      

    # Check for existing data
    #if not m.exists(inputfile):
    #    print(t)
    # uncomment the following to create data
    try:
        print(t)
        m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)

(1, 'shear', 'pointwise', 'upwind-advection', 'none')
Existing simulation da1be4...fa188f.nc
(1, 'shear', 'pointwise', 'upwind-advection', 'modal')
Existing simulation 790317...9cb678.nc
(1, 'shear', 'pointwise', 'centered-advection', 'none')
Existing simulation 6e5f48...124081.nc
(1, 'shear', 'pointwise', 'centered-advection', 'modal')
Existing simulation e9278f...b34355.nc
(1, 'shear', 'projection', 'upwind-advection', 'none')
Existing simulation eeb049...828c6b.nc
(1, 'shear', 'projection', 'upwind-advection', 'modal')
Existing simulation 563f85...5fa673.nc
(1, 'shear', 'projection', 'centered-advection', 'none')
Existing simulation 1b2356...e543e6.nc
(1, 'shear', 'projection', 'centered-advection', 'modal')
Existing simulation dedc56...a3f213.nc
(3, 'shear', 'pointwise', 'upwind-advection', 'none')
Existing simulation 6c32ac...dc570d.nc
(3, 'shear', 'pointwise', 'upwind-advection', 'modal')
Existing simulation 9d3732...ae52e9.nc
(3, 'shear', 'pointwise', 'centered-advection', 'none

In [11]:
# Try to vary timestepper
inputfile = generate_standard_input( 
    n=3, init="shear",multiplication="pointwise",
    advection="upwind-advection",regularization="none")  

inputfile["timestepper"]["tableau"] = "TVB-3-3"
print( "TVB-3-3")
try:
    m.create(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr)
inputfile["timestepper"]["tableau"] = "SSP-6-3"
print( "SSP-6-3")
try:
    m.create(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr )
 
inputfile["timestepper"]["type"] = "Shu-Osher"
inputfile["timestepper"]["tableau"] = "SSPRK-3-3"
inputfile["timestepper"]["dt"] *= 2
print( "SSPRK-3-3")
try:
    m.create(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr)

TVB-3-3
Existing simulation 64c937...daa7d2.nc
SSP-6-3
Existing simulation 7b8dcd...baa449.nc
SSPRK-3-3
Existing simulation b63509...50ae05.nc
